<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250709/%E5%89%B2%E5%BD%93%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 割当問題

### 準備


In [ ]:
%%capture
# python-mip ライブラリをインストールする
!pip install mip

# 数値計算ライブラリ numpy をインポートする
import mip
import numpy as np

---

## 演習問題1

### 1. 問題

3エージェント (A0, A1, A2) と3タスク (T0, T1, T2) の割当問題を以下のコスト行列で最適化し、最小の総コストを解答せよ。

$C = \begin{pmatrix} 10 & 8 & 12 \\ 7 & 11 & 9 \\ 9 & 4 & 10 \end{pmatrix}$

### 2. 数理モデルの定式化

この問題は、総コストを最小化する標準的な割当問題である。

**決定変数**

エージェント $i$ をタスク $j$ に割り当てる場合に 1、そうでない場合に 0 をとるバイナリ変数を $x_{ij}$ とする。 ($i, j \in \{0, 1, 2\}$)

**目的関数**

総割り当てコストを最小化するため、目的関数は以下のように表される。

$$\min Z = \sum_{i=0}^{2} \sum_{j=0}^{2} c_{ij} x_{ij}$$

ここで、$c_{ij}$ はコスト行列 $C$ の $(i, j)$ 成分である。

**制約条件**

1.  **エージェント割り当て制約**: 各エージェントは、正確に1つのタスクに割り当てられなければならない。
    $$\sum_{j=0}^{2} x_{ij} = 1, \quad \forall i \in \{0, 1, 2\}$$

2.  **タスク割り当て制約**: 各タスクは、正確に1人のエージェントに割り当てられなければならない。
    $$\sum_{i=0}^{2} x_{ij} = 1, \quad \forall j \in \{0, 1, 2\}$$

3.  **変数型制約**:
    $$x_{ij} \in \{0, 1\}, \quad \forall i, j \in \{0, 1, 2\}$$

### 3. Python (MIP) による実装

In [ ]:
# 演習問題1

# 問題データの設定
# エージェントとタスクの数
n = 3

# コスト行列
costs = np.array([
    [10, 8, 12],
    [7, 11, 9],
    [9, 4, 10]
])

# 1. モデルの作成
# sense=mip.MINIMIZE で最小化問題を指定
model1 = mip.Model(name="AssignmentProblem1", sense=mip.MINIMIZE)

# 2. 変数の定義
# x[i][j] はエージェントiをタスクjに割り当てるかどうかのバイナリ変数
x = [[model1.add_var(var_type=mip.BINARY, name=f"x({i},{j})") for j in range(n)] for i in range(n)]

# 3. 目的関数の設定
# 総コストを最小化する
model1.objective = mip.xsum(costs[i][j] * x[i][j] for i in range(n) for j in range(n))

# 4. 制約条件の追加
# 制約1: 各エージェントは1つのタスクに割り当てられる
for i in range(n):
    model1 += mip.xsum(x[i][j] for j in range(n)) == 1, f"AgentConstraint({i})"

# 制約2: 各タスクは1人のエージェントに割り当てられる
for j in range(n):
    model1 += mip.xsum(x[i][j] for i in range(n)) == 1, f"TaskConstraint({j})"

# 5. 問題の求解
status = model1.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小総コスト: {model1.objective_value:.2f}")
    print("最適な割り当て:")
    for i in range(n):
        for j in range(n):
            if x[i][j].x >= 0.99:
                print(f"  エージェント A{i} -> タスク T{j} (コスト: {costs[i][j]})")
else:
    print("最適解が見つかりませんでした。")

最小総コスト: 23.00
最適な割り当て:
  エージェント A0 -> タスク T0 (コスト: 10)
  エージェント A1 -> タスク T2 (コスト: 9)
  エージェント A2 -> タスク T1 (コスト: 4)


---

## 演習問題2

### 1. 問題

エージェント数とタスク数を4に増やし、以下のコスト行列で割当問題を解け。最小の総コストを解答せよ。

$C = \begin{pmatrix} 13 & 8 & 16 & 18 \\ 10 & 15 & 3 & 12 \\ 12 & 9 & 4 & 6 \\ 6 & 11 & 14 & 7 \end{pmatrix}$

### 2. 数理モデルの定式化

モデルの構造は演習問題1と同様であるが、エージェントとタスクの数が $n=4$ となる。

**決定変数**

$x_{ij} \in \{0, 1\}, \quad \forall i, j \in \{0, 1, 2, 3\}$

**目的関数**

$$\min Z = \sum_{i=0}^{3} \sum_{j=0}^{3} c_{ij} x_{ij}$$

**制約条件**

$$\sum_{j=0}^{3} x_{ij} = 1, \quad \forall i \in \{0, 1, 2, 3\}$$
$$\sum_{i=0}^{3} x_{ij} = 1, \quad \forall j \in \{0, 1, 2, 3\}$$

### 3. Python (MIP) による実装

In [ ]:
# 演習問題2

# 問題データの設定
# エージェントとタスクの数
n = 4

# コスト行列
costs = np.array([
    [13, 8, 16, 18],
    [10, 15, 3, 12],
    [12, 9, 4, 6],
    [6, 11, 14, 7]
])

# 1. モデルの作成
model2 = mip.Model(name="AssignmentProblem2", sense=mip.MINIMIZE)

# 2. 変数の定義
x = [[model2.add_var(var_type=mip.BINARY, name=f"x({i},{j})") for j in range(n)] for i in range(n)]

# 3. 目的関数の設定
model2.objective = mip.xsum(costs[i][j] * x[i][j] for i in range(n) for j in range(n))

# 4. 制約条件の追加
for i in range(n):
    model2 += mip.xsum(x[i][j] for j in range(n)) == 1, f"AgentConstraint({i})"
for j in range(n):
    model2 += mip.xsum(x[i][j] for i in range(n)) == 1, f"TaskConstraint({j})"

# 5. 問題の求解
status = model2.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小総コスト: {model2.objective_value:.2f}")
    print("最適な割り当て:")
    for i in range(n):
        for j in range(n):
            if x[i][j].x >= 0.99:
                print(f"  エージェント A{i} -> タスク T{j} (コスト: {costs[i][j]})")
else:
    print("最適解が見つかりませんでした。")

最小総コスト: 23.00
最適な割り当て:
  エージェント A0 -> タスク T1 (コスト: 8)
  エージェント A1 -> タスク T2 (コスト: 3)
  エージェント A2 -> タスク T3 (コスト: 6)
  エージェント A3 -> タスク T0 (コスト: 6)


---

## 演習問題3

### 1. 問題

3人の営業担当者 (S1, S2, S3) と3つの地域 (R1, R2, R3) があり、各担当者が各地域を担当した場合に期待される利益が以下の表で与えられている。総利益を最大にする割り当てを求め、総利益の最大値を解答せよ。

利益行列 $P = \begin{pmatrix} 50 & 65 & 70 \\ 80 & 75 & 60 \\ 55 & 60 & 75 \end{pmatrix}$

### 2. 数理モデルの定式化

この問題は、目的が「最大化」である点を除き、標準的な割当問題と同じである。

**決定変数**

担当者 $i$ を地域 $j$ に割り当てる場合に 1、そうでない場合に 0 をとるバイナリ変数を $x_{ij}$ とする。 ($i, j \in \{0, 1, 2\}$)

**目的関数**

総利益を最大化するため、目的関数は以下のように表される。

$$\max Z = \sum_{i=0}^{2} \sum_{j=0}^{2} p_{ij} x_{ij}$$

ここで、$p_{ij}$ は利益行列 $P$ の $(i, j)$ 成分である。

**制約条件**

制約条件は演習問題1と同一である。

### 3. Python (MIP) による実装

In [ ]:
# 演習問題3

# 問題データの設定
# 担当者と地域の数
n = 3

# 利益行列
profits = np.array([
    [50, 65, 70],
    [80, 75, 60],
    [55, 60, 75]
])

# 1. モデルの作成
# sense=mip.MAXIMIZE で最大化問題を指定
model3 = mip.Model(name="AssignmentProblem3_Maximize", sense=mip.MAXIMIZE)

# 2. 変数の定義
x = [[model3.add_var(var_type=mip.BINARY, name=f"x({i},{j})") for j in range(n)] for i in range(n)]

# 3. 目的関数の設定
# 総利益を最大化する
model3.objective = mip.xsum(profits[i][j] * x[i][j] for i in range(n) for j in range(n))

# 4. 制約条件の追加
for i in range(n):
    model3 += mip.xsum(x[i][j] for j in range(n)) == 1, f"AgentConstraint({i})"
for j in range(n):
    model3 += mip.xsum(x[i][j] for i in range(n)) == 1, f"TaskConstraint({j})"

# 5. 問題の求解
status = model3.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最大総利益: {model3.objective_value:.2f}")
    print("最適な割り当て:")
    for i in range(n):
        for j in range(n):
            if x[i][j].x >= 0.99:
                print(f"  担当者 S{i+1} -> 地域 R{j+1} (利益: {profits[i][j]})")
else:
    print("最適解が見つかりませんでした。")

最大総利益: 220.00
最適な割り当て:
  担当者 S1 -> 地域 R2 (利益: 65)
  担当者 S2 -> 地域 R1 (利益: 80)
  担当者 S3 -> 地域 R3 (利益: 75)


---

## 演習問題4

### 1. 問題

演習問題2で、エージェントをA0, A1, A2, A3、タスクをT0, T1, T2, T3とする。A1がT2を担当できないという制約を追加して、最適な割り当てを求め、総コストの最小値を解答せよ。

### 2. 数理モデルの定式化

この問題は、演習問題2のモデルに特定の割り当てを禁止する制約を追加したものである。

**追加制約**

エージェントA1 (インデックス $i=1$) はタスクT2 (インデックス $j=2$) を担当できない。これは、対応する決定変数 $x_{1,2}$ が 0 でなければならないことを意味する。

$$x_{1,2} = 0$$

この制約を演習問題2のモデルに追加する。

### 3. Python (MIP) による実装

In [ ]:
# 演習問題4

# 問題データの設定 (演習問題2と同じ)
n = 4
costs = np.array([
    [13, 8, 16, 18],
    [10, 15, 3, 12],
    [12, 9, 4, 6],
    [6, 11, 14, 7]
])

# 1. モデルの作成
model4 = mip.Model(name="AssignmentProblem4_Constrained", sense=mip.MINIMIZE)

# 2. 変数の定義
x = [[model4.add_var(var_type=mip.BINARY, name=f"x({i},{j})") for j in range(n)] for i in range(n)]

# 3. 目的関数の設定
model4.objective = mip.xsum(costs[i][j] * x[i][j] for i in range(n) for j in range(n))

# 4. 制約条件の追加
# 基本的な割当制約
for i in range(n):
    model4 += mip.xsum(x[i][j] for j in range(n)) == 1
for j in range(n):
    model4 += mip.xsum(x[i][j] for i in range(n)) == 1

# 追加制約: エージェントA1はタスクT2を担当できない
model4 += x[1][2] == 0, "A1_cannot_do_T2"

# 5. 問題の求解
status = model4.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小総コスト: {model4.objective_value:.2f}")
    print("最適な割り当て:")
    for i in range(n):
        for j in range(n):
            if x[i][j].x >= 0.99:
                print(f"  エージェント A{i} -> タスク T{j} (コスト: {costs[i][j]})")
else:
    print("最適解が見つかりませんでした。")

最小総コスト: 29.00
最適な割り当て:
  エージェント A0 -> タスク T1 (コスト: 8)
  エージェント A1 -> タスク T0 (コスト: 10)
  エージェント A2 -> タスク T2 (コスト: 4)
  エージェント A3 -> タスク T3 (コスト: 7)


---

## 演習問題5

### 1. 問題

3人のエージェントと4つのタスクがあるとする。各エージェントは1つのタスクを担当し、各タスクは高々1人のエージェントに担当される (つまり、タスクのうち1つは担当されない)。割り当てられたタスクの総コストの最小値を解答せよ。ただし、コスト行列 (3x4) は以下とする。

$C = \begin{pmatrix} 8 & 2 & 5 & 7 \\ 6 & 4 & 3 & 9 \\ 5 & 7 & 8 & 4 \end{pmatrix}$

### 2. 数理モデルの定式化

この問題はエージェントとタスクの数が異なる非均衡な (unbalanced) 割当問題である。

**決定変数**

エージェント $i$ をタスク $j$ に割り当てる場合に 1、そうでない場合に 0 をとるバイナリ変数を $x_{ij}$ とする。($i \in \{0, 1, 2\}, j \in \{0, 1, 2, 3\}$)

**目的関数**

$$\min Z = \sum_{i=0}^{2} \sum_{j=0}^{3} c_{ij} x_{ij}$$

**制約条件**

1.  **エージェント割り当て制約**: 各エージェントは、正確に1つのタスクに割り当てられなければならない。これは従来と同じである。
    $$\sum_{j=0}^{3} x_{ij} = 1, \quad \forall i \in \{0, 1, 2\}$$

2.  **タスク割り当て制約 (変更)**: 各タスクは、**高々1人 (at most one)** のエージェントに割り当てられる。つまり、割り当てられないタスクが存在する可能性がある。
    $$\sum_{i=0}^{2} x_{ij} \le 1, \quad \forall j \in \{0, 1, 2, 3\}$$

### 3. Python (MIP) による実装

In [ ]:
# 演習問題5

# 問題データの設定
# エージェントとタスクの数
num_agents = 3
num_tasks = 4

# コスト行列 (3x4)
costs = np.array([
    [8, 2, 5, 7],
    [6, 4, 3, 9],
    [5, 7, 8, 4]
])

# 1. モデルの作成
model5 = mip.Model(name="AssignmentProblem5_Unbalanced", sense=mip.MINIMIZE)

# 2. 変数の定義
x = [[model5.add_var(var_type=mip.BINARY, name=f"x({i},{j})") for j in range(num_tasks)] for i in range(num_agents)]

# 3. 目的関数の設定
model5.objective = mip.xsum(costs[i][j] * x[i][j] for i in range(num_agents) for j in range(num_tasks))

# 4. 制約条件の追加
# 制約1: 各エージェントは1つのタスクに割り当てられる
for i in range(num_agents):
    model5 += mip.xsum(x[i][j] for j in range(num_tasks)) == 1, f"AgentConstraint({i})"

# 制約2: 各タスクは高々1人のエージェントに割り当てられる (<= 1)
for j in range(num_tasks):
    model5 += mip.xsum(x[i][j] for i in range(num_agents)) <= 1, f"TaskConstraint({j})"

# 5. 問題の求解
status = model5.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小総コスト: {model5.objective_value:.2f}")
    print("最適な割り当て:")
    assigned_tasks = []
    for i in range(num_agents):
        for j in range(num_tasks):
            if x[i][j].x >= 0.99:
                print(f"  エージェント A{i} -> タスク T{j} (コスト: {costs[i][j]})")
                assigned_tasks.append(j)

    unassigned_task = set(range(num_tasks)) - set(assigned_tasks)
    print(f"担当されないタスク: T{list(unassigned_task)[0]}")

else:
    print("最適解が見つかりませんでした。")

最小総コスト: 9.00
最適な割り当て:
  エージェント A0 -> タスク T1 (コスト: 2)
  エージェント A1 -> タスク T2 (コスト: 3)
  エージェント A2 -> タスク T3 (コスト: 4)
担当されないタスク: T0


---

## 演習問題6

### 1. 問題

4人のエージェント (A0,A1,A2,A3)と4つのタスク (T0,T1,T2,T3)がある。コストは演習問題2の行列を使用する。制約として「もしエージェントA0がタスクT1に割り当てられたら、エージェントA1はタスクT3に割り当てられなければならない」を追加して最適化し、総コストの最小値を求めよ。

### 2. 数理モデルの定式化

この問題は、演習問題2のモデルに条件付きの論理制約を追加したものである。

**追加制約**

「もしエージェントA0がタスクT1に割り当てられたら ($x_{0,1}=1$)、エージェントA1はタスクT3に割り当てられなければならない ($x_{1,3}=1$)」

この論理命題は、$A \implies B$ の形であり、線形制約として $A \le B$ と表現できる。

$$x_{0,1} \le x_{1,3}$$

この制約を演習問題2のモデルに追加する。

### 3. Python (MIP) による実装

In [ ]:
# 演習問題6

# 問題データの設定 (演習問題2と同じ)
n = 4
costs = np.array([
    [13, 8, 16, 18],
    [10, 15, 3, 12],
    [12, 9, 4, 6],
    [6, 11, 14, 7]
])

# 1. モデルの作成
model6 = mip.Model(name="AssignmentProblem6_Logical", sense=mip.MINIMIZE)

# 2. 変数の定義
x = [[model6.add_var(var_type=mip.BINARY, name=f"x({i},{j})") for j in range(n)] for i in range(n)]

# 3. 目的関数の設定
model6.objective = mip.xsum(costs[i][j] * x[i][j] for i in range(n) for j in range(n))

# 4. 制約条件の追加
# 基本的な割当制約
for i in range(n):
    model6 += mip.xsum(x[i][j] for j in range(n)) == 1
for j in range(n):
    model6 += mip.xsum(x[i][j] for i in range(n)) == 1

# 追加制約: もし A0->T1 ならば A1->T3
# x(0,1) <= x(1,3)
model6 += x[0][1] <= x[1][3], "LogicalConstraint"

# 5. 問題の求解
status = model6.optimize()

# 6. 結果の表示
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小総コスト: {model6.objective_value:.2f}")
    print("最適な割り当て:")
    for i in range(n):
        for j in range(n):
            if x[i][j].x >= 0.99:
                print(f"  エージェント A{i} -> タスク T{j} (コスト: {costs[i][j]})")
else:
    print("最適解が見つかりませんでした。")

最小総コスト: 30.00
最適な割り当て:
  エージェント A0 -> タスク T1 (コスト: 8)
  エージェント A1 -> タスク T3 (コスト: 12)
  エージェント A2 -> タスク T2 (コスト: 4)
  エージェント A3 -> タスク T0 (コスト: 6)
